# 🎯 Agentic AI for PDE Discovery - Clean Version

**Optimized for Speed & Accuracy**

This notebook uses a multi-agent system to discover governing PDEs from spatiotemporal data.

## 📋 Quick Navigation
1. **Setup**: Dependencies, imports, configs
2. **Prompts**: All agent system messages (centralized)
3. **Data Loading**: Load and prepare PDE data
4. **Agent Definitions**: Create all agents
5. **Execution**: Run the discovery pipeline

---
## 1️⃣ SETUP & CONFIGURATION

In [1]:
# Install dependencies (run once)
!pip install -q autogen-agentchat autogen-ext pysindy scipy numpy matplotlib pillow


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Imports
import autogen
import numpy as np
import scipy.io as scio
import os
import base64
from pathlib import Path
from typing import Dict, List
from autogen import GroupChat, GroupChatManager, AssistantAgent, UserProxyAgent, Agent
from autogen_core import Image
import PIL
from autogen_agentchat.messages import MultiModalMessage

print("✅ All imports successful")

✅ All imports successful


In [ ]:
# Configuration: ADJUST THESE FOR SPEED/ACCURACY TRADE-OFF
CONFIG = {
    # Speed settings
    "n_x_sub": 512,        # Spatial resolution (higher = more accurate, slower)
    "n_t_sub": 256,        # Temporal resolution (higher = more accurate, slower)
    "max_rounds": 30,      # Max iterations (lower = faster, may miss solution)
    
    # Accuracy settings
    "l2_threshold": 0.01,  # Acceptance threshold (lower = stricter)
    "dataset_name": "KS",  # Dataset to analyze
    
    # API settings
    "llm_model": "qwen/qwen3-coder-480b-a35b-instruct",
    "vlm_model": "microsoft/phi-4-multimodal-instruct",
    "api_key": "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",
    "base_url": "https://integrate.api.nvidia.com/v1",
}

# Build configs
llm_config = {
    "config_list": [{
        "model": CONFIG["llm_model"],
        "base_url": CONFIG["base_url"],
        "api_key": CONFIG["api_key"],
    }],
    "timeout": 120,
}

vlm_config = {
    "config_list": [{
        "model": CONFIG["vlm_model"],
        "base_url": CONFIG["base_url"],
        "api_key": CONFIG["api_key"],
        "price": [0, 0]
    }],
    "timeout": 120,
}

print(f" Config loaded: {CONFIG['n_x_sub']}x{CONFIG['n_t_sub']} resolution, {CONFIG['max_rounds']} max rounds")

⚙️ Config loaded: 512x256 resolution, 30 max rounds


In [4]:
# Define symbol library for PDEs
def library(P, D):
    """Generates operands and operators based on max Poly order P and Deriv order D."""
    operands = []
    operators = []
    for i in range(D + 1): 
        operands.append("u" if i==0 else "u_" + "x" * i)
    operands.append("x")
    for j in range(2, P + 1): 
        operators.append("**" + f"{j}")
    for op in ["/", "*", "+", "-"]: 
        operators.append(op)
    return operands, operators

operands, operators = library(4, 4)
print(f"📚 Symbol library: {operands}")
print(f"🔧 Operators: {operators}")

📚 Symbol library: ['u', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx', 'x']
🔧 Operators: ['**2', '**3', '**4', '/', '*', '+', '-']


---
## 2️⃣ CENTRALIZED AGENT PROMPTS

All agent system messages defined here for easy editing.

In [5]:
# === AGENT PROMPTS ===

PROMPTS = {
    "vlm": """
You are a vision-language scientist analyzing spatiotemporal plots.

TASK: Analyze contour and surface plots of u(x,t) to identify:
1. Axes, ranges, and color scales
2. Qualitative patterns:
   - Shocks (tight contours) vs smooth regions
   - Traveling waves (slanted bands)
   - Periodicity and symmetry
3. Derivative signatures:
   - Contour spacing → |∇u| (u_x, u_t)
   - Curvature → u_xx (diffusion)
   - Ripples/oscillations → u_xxxx (dispersion)
   - Amplitude-gradient coupling → u*u_x (nonlinearity)
4. Boundary behavior

Be concise but specific.
""",

    "llm": f"""
You are a mathematical physicist specializing in PDE discovery.

ROLE: Hypothesis Generator (Symbolic PDEs)

INPUT: Visual analysis from VLM + Feedback from Critic (if iterating)

TASK:
1. Map visual cues to PDE terms:
   - "Dispersion" → u_xxxx, u_xxx
   - "Nonlinear advection" → u*u_x
   - "Diffusion" → u_xx
   - "Reaction" → u, u**2

2. Generate 10 candidate PDEs of form: u_t = ...
   - Include known templates (Burgers, KS, KdV, Fisher-KPP)
   - Explore novel combinations
   - Ensure physical plausibility

3. Use ONLY these symbols:
   - Operands: {operands}
   - Operators: {operators}

CONSTRAINTS:
- NO code generation
- NO explanations
- Output EXACTLY 10 equations (one per line)

EXAMPLE OUTPUT:
u_t = -u*u_x - u_xx - u_xxxx
u_t = u_xx + u - u**3
...

If receiving Critic feedback, incorporate suggestions (e.g., "add u_xxxx for dispersion").
""",

    "engineer": """
You are an Engineering Agent that evaluates candidate PDEs using PySINDy.

INPUT: 10 PDEs from LLM (e.g., u_t = -u*u_x - u_xx - u_xxxx)

TASK:
1. Load data from: u.npy, x.npy, t.npy
2. Compute derivatives using PySINDy (u_t, u_x, u_xx, u_xxx, u_xxxx)
3. For each PDE:
   a. Parse RHS terms
   b. Fit coefficients via least-squares
   c. Compute Relative L2 Error:
      error = ||u_t_true - u_t_pred|| / ||u_t_true||
4. Save results:
   - score.csv (L2 errors)
   - error.csv (spatial error fields)
   - boundaries.csv (boundary values)
5. Print fitted PDEs:
   Format: "Fitted PDE: u_t = c1*term1 + c2*term2 + ... : score = X.XXXX"
   (coefficients rounded to 4 decimals)

RULES:
- Write ONE complete Python script (no partial code)
- Use ONLY provided data (no synthetic data)
- Save script as pde_sim.py
""",

    "scientist": """
You are a Scientist reviewing PDE evaluation results.

ROLE: Quality Control

TASK:
1. Check execution:
   - exitcode: 0? Files saved? Scores printed?
2. Validate PDEs:
   - Physically meaningful terms?
   - Reasonable coefficients?
   - Good fit quality?

SIGNALS:
- If success + valid → End with: <<SIGNAL: ROGER CRITIC>>
- If code errors → End with: <<SIGNAL: ROGER ENGINEER>>

L2 ERROR RUBRIC:
< 0.01: Excellent
0.01-0.03: Good
0.03-0.05: Acceptable
> 0.05: Poor

Provide clear feedback to Engineer if code fails.
""",

    "critic": f"""
You are the Critic - final decision maker for PDE discovery.

GOAL: Find the TRUE governing PDE (not just best fit).

INPUT:
- 10 fitted PDEs with scores
- Scientist's validation

DECISION LOGIC:
IF any PDE meets ALL criteria:
  1. L2 Error ≤ {CONFIG['l2_threshold']}
  2. Mathematically valid (well-posed, consistent)
  3. Physically meaningful (terms make sense)
  4. Stable boundaries
THEN:
  → Signal: <<SIGNAL: TERMINATE>>
  → Declare victory!

ELSE:
  → Analyze WHY all PDEs failed
  → Generate specific feedback for LLM:
     Examples:
     - "High errors suggest missing u_xxxx for dispersion"
     - "Coefficient on u*u_x too large; reduce nonlinearity"
     - "Try adding damping term u_xx"
  → Signal: <<SIGNAL: ROGER LLM>>

EVALUATION:
- Check ALL PDEs (not just lowest score)
- Reject if unphysical (e.g., u_t = x**5)
- Be strict but fair

Only terminate when CONFIDENT in the result.
""",
}

print("✅ All prompts loaded (5 agents)")

✅ All prompts loaded (5 agents)


---
## 3️⃣ DATA LOADING

In [6]:
# Data loading function
def load_pde_data(dataset_name, n_x_sub=512, n_t_sub=256):
    """Load and subsample PDE data."""
    base_path = os.path.join(os.getcwd(), 'Bayesian_PDE_Discovery_DATA')
    print(f"\n📂 Loading: {dataset_name}")
    
    # Load data based on dataset
    if dataset_name == 'KS':
        fpath = os.path.join(base_path, 'kuramoto_sivishinky.mat')
        data = scio.loadmat(fpath)
        u = data['u']
        x = data['x'].flatten()
        t = data['t'].flatten()
    elif dataset_name == 'Burgers':
        fpath = os.path.join(base_path, 'burgers.mat')
        data = scio.loadmat(fpath)
        u = data['usol']
        x = data['x'].flatten()
        t = data['t'].flatten()
    else:
        raise ValueError(f"Unknown dataset: {dataset_name}")
    
    # Subsample
    n_x, n_t = u.shape
    n_x_sub = min(n_x_sub, n_x)
    n_t_sub = min(n_t_sub, n_t)
    
    x_idx = np.linspace(0, n_x-1, n_x_sub, dtype=int)
    t_idx = np.linspace(0, n_t-1, n_t_sub, dtype=int)
    
    u_sub = u[np.ix_(x_idx, t_idx)]
    x_sub = x[x_idx]
    t_sub = t[t_idx]
    
    print(f"✅ Loaded: u={u_sub.shape}, x={x_sub.shape}, t={t_sub.shape}")
    return u_sub, x_sub, t_sub

# Load and save data
u_full, x_full, t_full = load_pde_data(
    CONFIG["dataset_name"], 
    CONFIG["n_x_sub"], 
    CONFIG["n_t_sub"]
)

np.save("u.npy", u_full)
np.save("x.npy", x_full)
np.save("t.npy", t_full)

print(f"💾 Saved to: u.npy, x.npy, t.npy")


📂 Loading: KS
✅ Loaded: u=(512, 251), x=(512,), t=(251,)
💾 Saved to: u.npy, x.npy, t.npy


---
## 4️⃣ AGENT DEFINITIONS

In [7]:
# Termination function
def termination_message(msg):
    return "<<SIGNAL: TERMINATE>>" in str(msg.get("content", ""))

# User Proxy (Admin)
user_proxy = autogen.UserProxyAgent(
    name="Admin",
    system_message="You are a human admin.",
    is_termination_msg=termination_message,
    human_input_mode="NEVER",
    code_execution_config=False,
)

# VLM (Vision Analyzer)
vlm = autogen.AssistantAgent(
    name="Contour_Plot_Analyser",
    llm_config=vlm_config,
    system_message=PROMPTS["vlm"]
)

# LLM (PDE Generator)
llm = autogen.AssistantAgent(
    name="Llm",
    llm_config=llm_config,
    system_message=PROMPTS["llm"]
)

# Engineer (Code Generator)
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    system_message=PROMPTS["engineer"]
)

# Executor (Code Runner)
executor = autogen.UserProxyAgent(
    name="Executor",
    system_message="Execute code and report results.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": ".", "use_docker": False},
)

# Scientist (Validator)
scientist = autogen.AssistantAgent(
    name="Scientist",
    llm_config=llm_config,
    system_message=PROMPTS["scientist"]
)

# Critic (Decision Maker)
critic = autogen.AssistantAgent(
    name="Critic",
    llm_config=llm_config,
    system_message=PROMPTS["critic"]
)

print("✅ All agents created (7 total)")

✅ All agents created (7 total)


In [8]:
# Speaker selection function (agent routing)
def custom_speaker_selection(last_speaker: Agent, groupchat):
    """Route conversation between agents."""
    messages = groupchat.messages
    
    if len(messages) == 0:
        return vlm
    
    last_msg = messages[-1]["content"]
    
    # Routing logic
    if last_speaker.name == "Contour_Plot_Analyser":
        return llm
    elif last_speaker.name == "Llm":
        return engineer
    elif last_speaker.name == "Engineer":
        return executor
    elif last_speaker.name == "Executor":
        return scientist
    elif last_speaker.name == "Scientist":
        if "<<SIGNAL: ROGER ENGINEER>>" in str(last_msg):
            return engineer
        else:
            return critic
    elif last_speaker.name == "Critic":
        if "<<SIGNAL: TERMINATE>>" in str(last_msg):
            return user_proxy
        else:
            return llm
    else:
        return groupchat.agents[(groupchat.agents.index(last_speaker) + 1) % len(groupchat.agents)]

print("✅ Speaker selection function ready")

✅ Speaker selection function ready


---
## 5️⃣ EXECUTION

In [9]:
# Prepare multimodal message with plots
def create_seed_message(dataset_name):
    """Create initial message with contour/surface plots."""
    plot_dir = "surface_contour_plots"
    contour_path = os.path.join(plot_dir, f"{dataset_name}_contour_896.png")
    surface_path = os.path.join(plot_dir, f"{dataset_name}_surface_896.png")
    
    content = [{"type": "input_text", "text": "Analyze these plots for PDE discovery."}]
    
    for path in [contour_path, surface_path]:
        if os.path.exists(path):
            b64 = base64.b64encode(Path(path).read_bytes()).decode("utf-8")
            content.append({
                "type": "image_url", 
                "image_url": {"url": f"data:image/png;base64,{b64}", "detail": "high"}
            })
    
    return {"role": "user", "content": content}

seed_msg = create_seed_message(CONFIG["dataset_name"])
print(f"📸 Loaded plots for {CONFIG['dataset_name']}")

📸 Loaded plots for KS


In [10]:
# Create group chat
groupchat = autogen.GroupChat(
    agents=[user_proxy, vlm, llm, engineer, executor, scientist, critic],
    messages=[],
    max_round=CONFIG["max_rounds"],
    speaker_selection_method=custom_speaker_selection,
)

manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

print(f"🎯 Group chat ready (max {CONFIG['max_rounds']} rounds)")

🎯 Group chat ready (max 30 rounds)


In [11]:
# 🚀 RUN THE SYSTEM
print("\n" + "="*60)
print("🚀 STARTING PDE DISCOVERY PIPELINE")
print("="*60 + "\n")

groupchat.reset()
result = user_proxy.initiate_chat(manager, message=seed_msg)

print("\n" + "="*60)
print("✅ PIPELINE COMPLETE")
print("="*60)


🚀 STARTING PDE DISCOVERY PIPELINE

Admin (to chat_manager):

Analyze these plots for PDE discovery.
<image>
<image>

--------------------------------------------------------------------------------

Next speaker: Contour_Plot_Analyser

Contour_Plot_Analyser (to chat_manager):

It appears that you have described two images, but since these descriptions are text-only, I'm unable to directly analyze images. However, I can certainly help guide you through the process of discovering Partial Differential Equations (PDEs) from plots of solutions.

When examining plots to discover PDEs, you should consider:

1. **Identifying the Type of PDE**: Determine if the equation is linear or nonlinear, and whether it's elliptic, parabolic, or hyperbolic. This classification helps in understanding the general form of the PDE and the techniques required to solve it.

2. **Examine Boundary and Initial Conditions**: Investigate any given boundary and/or initial conditions that are represented in the plot. 

---
## 📊 RESULTS ANALYSIS

In [12]:
# Check results
import pandas as pd

if os.path.exists("score.csv"):
    scores = pd.read_csv("score.csv")
    print("\n📈 PDE Scores:")
    print(scores)
    
    best_idx = scores['score'].idxmin()
    print(f"\n🏆 Best PDE (index {best_idx}): L2 Error = {scores.loc[best_idx, 'score']:.6f}")
else:
    print("⚠️ No score.csv found - check execution logs")


📈 PDE Scores:
   9.999903570504926620e-01
0                  0.999991
1                  0.999997
2                  0.999997
3                  0.999994
4                  0.999994
5                  1.000000
6                  1.000000
7                  0.999990
8                  0.999997


KeyError: 'score'